In [1]:
!pip install google-cloud-storage

In [2]:
from google.colab import drive, auth
drive.mount('/content/drive')
auth.authenticate_user()

# Establece las credenciales para Google Cloud
!gcloud auth application-default login

Mounted at /content/drive

You are running on a Google Compute Engine virtual machine.
The service credentials associated with this virtual machine
will automatically be used by Application Default
Credentials, so it is not necessary to use this command.

If you decide to proceed anyway, your user credentials may be visible
to others with access to this virtual machine. Are you sure you want
to authenticate with your personal account?

Do you want to continue (Y/n)?  n

ERROR: (gcloud.auth.application-default.login) Aborted by user.


In [3]:
import requests
import pandas as pd

In [4]:
# URL de la API de Harry Potter
API_URL = "https://hp-api.onrender.com/api/characters"

# Conexión a la API
try:
    response = requests.get(API_URL)
    response.raise_for_status()
    data = response.json()
    print("Conexión exitosa")
except requests.exceptions.RequestException as e:
    print(f"Error al conectarse a la API: {e}")
    data = []

Conexión exitosa


In [5]:
# Procesar datos
characters = []
for char in data:
    role = "Staff" if char.get("hogwartsStaff") else ("Student" if char.get("hogwartsStudent") else "Unknown")
    character = {
        "name": char.get("name"),
        "house": char.get("house", "Unknown"),
        "role": role
    }
    characters.append(character)


In [6]:
# Crear DataFrame
df = pd.DataFrame(characters)

# Mostrar los primeros 5 registros
print(df.head())

                 name       house     role
0        Harry Potter  Gryffindor  Student
1    Hermione Granger  Gryffindor  Student
2         Ron Weasley  Gryffindor  Student
3        Draco Malfoy   Slytherin  Student
4  Minerva McGonagall  Gryffindor    Staff


In [7]:

output_file = "/content/drive/My Drive/harry_potter_characters.csv"
df.to_csv(output_file, index=False, encoding="utf-8")
print(f"Datos guardados en {output_file}")

Datos guardados en /content/drive/My Drive/harry_potter_characters.csv


In [8]:
from google.cloud import storage

def upload_csv_to_gcs(bucket_name, source_file_name, destination_blob_name):

    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)

    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(f"Archivo {source_file_name} subido a {destination_blob_name} en el bucket {bucket_name}.")

output_file = "harry_potter_characters.csv"

df.to_csv(output_file, index=False, encoding="utf-8")

bucket_name = "bucket_final_bd"
destination_blob_name = "harry_potter_characters.csv"

upload_csv_to_gcs(bucket_name, output_file, destination_blob_name)


Archivo harry_potter_characters.csv subido a harry_potter_characters.csv en el bucket bucket_final_bd.
